In [1]:
from tensorboard.backend.event_processing import event_accumulator
import os
import numpy as np
import pandas as pd
import json

def load_metric_from_event(event_name): 
    event_path = os.path.join("runs", event_name)
    event_name = os.listdir(event_path)[0]
    event_path = os.path.join(event_path, event_name)

    # only read scalars
    ea = event_accumulator.EventAccumulator(event_path, size_guidance={event_accumulator.SCALARS: 0})

    # load scalars
    ea.Reload()

    return get_best_epoch_metric(ea, metric)

def get_best_epoch_metric(ea, metric):
    test_metrics = ea.Scalars(f"test/{metric}")
    test_metric_vals = [metric.value for metric in test_metrics]
    test_best_metric_idx = np.argmax(test_metric_vals)
    test_best_metric = np.max(test_metric_vals)
    test_best_epoch = test_metrics[test_best_metric_idx].step

    train_metrics = ea.Scalars(f"train/{metric}")
    train_best_metric = train_metrics[test_best_metric_idx].value
    
    return [test_best_epoch, train_best_metric, test_best_metric]

## Define important variables

In [2]:
# events in tensorboard
events = ["23_03_2021_21_46_51_task_standard_seed_7"]
metric = "acc"
# choose which columsn to include
include_cols = ["task", "ic_loss_weight", "num_imgs", "same_class_prob", "seed"]

In [3]:
results = []
args_dir = os.path.join("args")
for event_name in events:
    # get parameters
    with open(f"{args_dir}\\args_{event_name}.json") as json_file:
        params = json.load(json_file)
    # wrap each value into a list
    params = {key:[value] for key, value in params.items()}

    # create dataframe and add parameters
    results_df = pd.DataFrame(params)[include_cols]

    # add metrics
    metric_cols = ["best epoch", f"best train {metric}", f"best test {metric}"]
    results_df.loc[:, metric_cols] = load_metric_from_event(event_name)

    # append to final table
    results.append(results_df)

results = pd.concat(results)

In [4]:
results

,task,ic_loss_weight,num_imgs,same_class_prob,seed,best epoch,best train acc,best test acc
0,standard,1.0,2,0.0,7,3,0.96814,0.9756
